# The feedforward neural network model for learning PageRank

In [1]:
import time
from helpers import *

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [3]:
# Pageank as the target or label data
target_pagerank = np.load("fb_co_pages_pageranks.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_pagerank[0]

array([9.98507331e-05])

In [4]:
# scaling the PageRank data
target_pagerank = target_pagerank * 10000
target_pagerank[0]

array([0.99850733])

In [5]:
target_pagerank.shape

(14113, 1)

In [6]:
# the order 5 NFDC matrix as the feature set
matrix_1 = np.load("fb_co_pages_discounted_NDFC_matrix_r1-30_sta1_max50_rad5.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [7]:
matrix_1.shape

(14113, 6, 17)

In [8]:
matrix_1[111]

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, 2.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 0.00000e+00, 2.05556e-01, 3.27778e-01, 8.33330e-02,
        0.00000e+00, 2.05556e-01, 5.55560e-02, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.22222e-01,
        0.00000e+00, 0.00000e+00],
       [1.91670e-02, 1.25000e-03, 5.37500e-02, 1.47197e-01, 1.47917e-01,
        9.60980e-02, 6.51140e-02, 1.86780e-01, 3.08330e-02, 5.54170e-02,
        2.45830e-02, 8.75000e-03, 2.04170e-02, 6.15150e-02, 3.15150e-02,
        1.81820e-02, 3.15150e-02],
       [1.14010e-02, 2.27700e-02, 2.77640e-02, 6.91750e-02, 5.25770e-02,
        6.92940e-02, 9.92490e-02, 9.42720e-02, 9.63990e-02, 1.23248e-01,
        6.29110e-02, 4.95400e-02, 3.59460e-02, 2.51710e-02, 1.24235e-01,
        1.33000e-02

In [9]:
matrix_1 = matrix_1.reshape(14113,-1)
matrix_1.shape

(14113, 102)

# Converting the data into pytorch tenors

In [10]:
features = torch.from_numpy(matrix_1)
targets = torch.from_numpy(target_pagerank)
features.shape, targets.shape

(torch.Size([14113, 102]), torch.Size([14113, 1]))

In [11]:
# Activating the automatic gradient 
features.requires_grad_(True)
targets.requires_grad_(True)

tensor([[0.9985],
        [0.8875],
        [0.5463],
        ...,
        [0.2811],
        [0.4245],
        [0.3886]], dtype=torch.float64, requires_grad=True)

In [12]:
# Shuffling and dividing the indecies
n_samples = features.shape[0]
n_test = 4113
shuffled_ind = torch.randperm(n_samples)
train_ind = shuffled_ind[:-n_test]
test_ind = shuffled_ind[-n_test:]
# Dividing features and targets into tain and test sets
train_features = features[train_ind]
test_features = features[test_ind]
train_targets = targets[train_ind]
test_targets = targets[test_ind]
train_features.shape, test_features.shape, train_targets.shape, test_targets.shape

(torch.Size([10000, 102]),
 torch.Size([4113, 102]),
 torch.Size([10000, 1]),
 torch.Size([4113, 1]))

## A function for dividing train data into batches

In [13]:
# dividing train_features and train_targets into batches
def next_batch(train_features, train_targets, batch_size=100):
    num_features = train_features.shape[0]
    # Shuffling
    shuffled_ind = torch.randperm(num_features)
    shuffled_train_features = train_features[shuffled_ind]
    shuffled_train_targets = train_targets[shuffled_ind]
    # Dividing
    i = 0
    while i < num_features:
        i += batch_size
        yield (shuffled_train_features[i-batch_size:i], shuffled_train_targets[i-batch_size:i])
    return

## The feedforward neural network model

In [14]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        num_features = features.shape[1]
        self.fc1 = nn.Linear(num_features, 400)
        self.fc2 = nn.Linear(400, 800)
        self.fc3 = nn.Linear(800, 200)
        self.fc4 = nn.Linear(200, 64)
        self.fc5 = nn.Linear(64, 8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
    
    def forward(self, X):
        X = torch.tanh(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc3(X))
        X = self.dropout3(X)
        X = torch.relu(self.fc4(X))
        X = self.dropout2(X)
        X = torch.tanh(self.fc5(X))
        return self.fc6(X)

In [15]:
# Instantiation of the model
torch.manual_seed(42)
model = FFNN_model()
model

FFNN_model(
  (fc1): Linear(in_features=102, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
)

In [16]:
# Number of parameters
num_para = 0
for param in model.parameters():
    print(param.numel())
    num_para += param.numel()
print("----------------------")
print(f'Number of all parameters: \n{num_para}')

40800
400
320000
800
160000
200
12800
64
512
8
8
1
----------------------
Number of all parameters: 
535593


In [17]:
# Objective function and optimmizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Defining the Training Loop

In [18]:
def training_loop(n_epochs=1000,
                  batch_size=100,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets):
    num_features = train_features.shape[0]
    start_time = time.time()
    all_train_loss, all_test_loss = np.zeros(n_epochs), np.zeros(n_epochs)
    for epoch in range(1, n_epochs +1):
        # Training: 
        epoch_losses = []
        # looping through batches
        for train_features, train_targets in next_batch(train_features=train_features, 
                                                        train_targets=train_targets, batch_size=batch_size): 
            train_preds = model(train_features.float())
            train_loss = loss_fn(train_targets.float(), train_preds.float())
            optimizer.zero_grad()
            train_loss.backward(retain_graph=True)
            optimizer.step()
            
            epoch_losses.append(train_loss.item())
        average_epoch_loss = sum(epoch_losses)/len(epoch_losses)
        
        # Test:
        with torch.no_grad():
            test_preds = model(test_features.float())
            test_loss = loss_fn(test_targets.float(), test_preds.float())
        
        all_train_loss[epoch - 1] = average_epoch_loss
        all_test_loss[epoch - 1] = test_loss.item()
        # Printing the result: 
        if epoch == 1 or epoch % 100 == 0:
            print(f"EPOCH: {epoch:{7}}")
            print(f"MEAN TRAIN LOSS:   {average_epoch_loss:.11f},    TEST LOSS:   {test_loss.item():.11f}")
            print("-----------------------------------------")
    print("The total time = ", np.round(time.time() - start_time, 3), " seconds!")
    return all_train_loss, all_test_loss

# Training the model

In [19]:
losses = training_loop(n_epochs=2000,
                  batch_size=400,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets)

EPOCH:       1
MEAN TRAIN LOSS:   0.57362449408,    TEST LOSS:   0.39788141847
-----------------------------------------
EPOCH:     100
MEAN TRAIN LOSS:   0.14499205351,    TEST LOSS:   0.25034418702
-----------------------------------------
EPOCH:     200
MEAN TRAIN LOSS:   0.09449286759,    TEST LOSS:   0.19527198374
-----------------------------------------
EPOCH:     300
MEAN TRAIN LOSS:   0.06507074833,    TEST LOSS:   0.16484817863
-----------------------------------------
EPOCH:     400
MEAN TRAIN LOSS:   0.04731440544,    TEST LOSS:   0.14349725842
-----------------------------------------
EPOCH:     500
MEAN TRAIN LOSS:   0.03464121744,    TEST LOSS:   0.13228189945
-----------------------------------------
EPOCH:     600
MEAN TRAIN LOSS:   0.02549823560,    TEST LOSS:   0.12769885361
-----------------------------------------
EPOCH:     700
MEAN TRAIN LOSS:   0.02140236832,    TEST LOSS:   0.11747862399
-----------------------------------------
EPOCH:     800
MEAN TRAIN LOSS: 

In [20]:
model.eval()
errors = []
for i in range(n_test):
    targ = test_targets[i].item()
    feat = test_features[i].float().view(1,1,102)
    pred = model(feat).item()
    inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%100 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------------------------------------")

target 0.4980338141461339,    prediction: 0.5440704226493835
index 0:       inaccuracy: 9.244%
-----------------------------------------------------------------
target 0.5008044540971669,    prediction: 0.5267464518547058
index 100:       inaccuracy: 5.18%
-----------------------------------------------------------------
target 1.191392550177932,    prediction: 1.3120272159576416
index 200:       inaccuracy: 10.126%
-----------------------------------------------------------------
target 0.49278578777108006,    prediction: 0.5346738696098328
index 300:       inaccuracy: 8.5%
-----------------------------------------------------------------
target 0.3929430916496587,    prediction: 0.38885530829429626
index 400:       inaccuracy: 1.04%
-----------------------------------------------------------------
target 1.353300532844947,    prediction: 1.270524263381958
index 500:       inaccuracy: 6.117%
-----------------------------------------------------------------
target 0.7172099031383945,  

In [21]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  9.09


In [22]:
torch.save(model, "trained_FFNN_model_pagerank_discounted.pt")